<a href="https://colab.research.google.com/github/KozhevnikovAlexandr/zootopia-project/blob/main/vgg16_zootopia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.optimizers import SGD
from google.colab import files
from IPython.display import Image
import numpy as np
from google.colab import drive
import zipfile
import shutil
from keras import losses
import os
from keras.optimizers import Adam

In [63]:
# Каталог с данными для обучения
train_dir = '/content/train'
# Каталог с данными для проверки
val_dir = '/content/test'
# Каталог с данными для тестирования
test_dir = '/content/val'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (img_width, img_height, 3)
# Размер мини-выборки 
batch_size = 100
# Количество изображений для обучения
nb_train_samples = 2000 
# Количество изображений для проверки
nb_validation_samples = 100
# Количество изображений для тестирования
nb_test_samples = 100
#Количество эпох
epochs_count = 10
#Количество нейронов на полносвязном слое
dense_count = 512

In [56]:
#подключение гугл диска
!mkdir /content/drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
#распаковка архивов
zip_file = '/content/drive/My Drive/set.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

In [23]:
zip_file = '/content/drive/My Drive/sorted.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

In [ ]:
zip_file = '/content/drive/My Drive/work_test.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

In [30]:
!mkdir /content/train
!mkdir /content/train/phone
!mkdir /content/train/no_phone
!mkdir /content/val
!mkdir /content/val/phone
!mkdir /content/val/no_phone
!mkdir /content/test
!mkdir /content/test/phone
!mkdir /content/test/no_phone

In [31]:
#функция пермещает и соритрует фото по каталогам для генераторов
def move_files(from_dir, to_train, to_val, to_test, format, train_count):
  val_count = 0
  test_count = 0
  current_train_count = 0
  for root, dirs, files in os.walk(from_dir):
    for file in files:
      if (file.endswith(format)):
        if val_count < nb_validation_samples // 2:
          val_count += 1
          os.rename(os.path.join(from_dir, file), os.path.join(to_val, file))
          continue
        if test_count < nb_test_samples // 2:
          test_count += 1
          os.rename(os.path.join(from_dir, file), os.path.join(to_test, file)) 
          continue
        if current_train_count < train_count:
          current_train_count += 1
          os.rename(os.path.join(from_dir, file), os.path.join(to_train, file))
          continue

In [32]:
move_files('/content/set/', '/content/train/phone/', '/content/val/phone/', '/content/test/phone/', '.jpg', 1000)

In [33]:
move_files('/content/sorted/', '/content/train/no_phone/', '/content/val/no_phone/', '/content/test/no_phone/', '.JPEG', 1000)

In [65]:
#инициализация готовой части сети
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
#отключение обучения уже обученной части
vgg16_net.trainable = False

In [76]:
#инициализация сети, добавление слоев
model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(dense_count))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [67]:
#компиляция модели
#фунуция ошибок - среднекватратичная 
#оптимизатор Adam
model.compile(loss=losses.mean_squared_error,
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [68]:
#создание генератора, помещаем все пиксели изображения в диапазон от 0 до 255
datagen = ImageDataGenerator(rescale=1. / 255)

In [69]:
#генератор данных для обучения
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1996 images belonging to 2 classes.


In [70]:
#генератор данных для проверки
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 100 images belonging to 2 classes.


In [71]:
#генератор данных для тестиования
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 100 images belonging to 2 classes.


In [74]:
#генератор рабочих данных для тестиования
work_generator = datagen.flow_from_directory(
    '/content/work_test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 164 images belonging to 2 classes.


In [72]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs_count,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
20/20 [==============================] - 464s 23s/step - loss: 0.3323 - accuracy: 0.5416 - val_loss: 0.1521 - val_accuracy: 0.9200
Epoch 2/10
20/20 [==============================] - 458s 23s/step - loss: 0.1558 - accuracy: 0.7962 - val_loss: 0.0725 - val_accuracy: 0.9900
Epoch 3/10
20/20 [==============================] - 457s 23s/step - loss: 0.0815 - accuracy: 0.9152 - val_loss: 0.0431 - val_accuracy: 1.0000
Epoch 4/10
20/20 [==============================] - 456s 23s/step - loss: 0.0542 - accuracy: 0.9576 - val_loss: 0.0258 - val_accuracy: 1.0000
Epoch 5/10
20/20 [==============================] - 458s 23s/step - loss: 0.0372 - accuracy: 0.9692 - val_loss: 0.0186 - val_accuracy: 1.0000
Epoch 6/10
20/20 [==============================] - 461s 23s/step - loss: 0.0309 - accuracy: 0.9753 - val_loss: 0.0138 - val_accuracy: 1.0000
Epoch 7/10
20/20 [==============================] - 459s 23s/step - loss: 0.0238 - accuracy: 0.9851 - val_loss: 0.0103 - val_accuracy: 1.0000
Epoch 

In [73]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных (синтетика): %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность на тестовых данных: 100.00%


In [75]:
scores = model.evaluate_generator(work_generator, 164 // batch_size)
print("Точность на рабочих данных (с вебки): %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность на рабочих данных: 33.00%
